In [1]:
import cv2
import numpy as np

In [26]:
def capture_video(apply_func=None, *args):
    cap = cv2.VideoCapture(0)
    while True:
            # Capture frame-by-frame
            ret, frame = cap.read()

            # Our operations on the frame come here
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            if apply_func is not None:
                display_frame = apply_func(frame, *args)
            else:
                display_frame = frame
                
            # Display the resulting frame
            cv2.imshow('frame', display_frame)
            # When everything done, release the capture
            if cv2.waitKey(1) & 0xFF == ord('q'):
                    break

    cap.release()
    cv2.destroyAllWindows()

In [36]:
def gray(img):
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [37]:
capture_video(gray)

In [30]:
def sobel_vertical(img):
    return cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=5)

In [31]:
capture_video(sobel_vertical)

In [32]:
def sobel_horizontal(img):
    return cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=5)

In [33]:
capture_video(sobel_horizontal)

In [34]:
def sobel_both(img):
    return cv2.Sobel(img, cv2.CV_64F, 1, 1, ksize=5)

In [50]:
capture_video(sobel_both)

In [47]:
def edges(img):
    return cv2.Canny(img, 100, 150)

In [48]:
capture_video(edges)